In [36]:
from bs4 import BeautifulSoup
import requests

In [37]:
url_template = "https://www.nwac.us/avalanche-forecast/current/{region}"
ex_region = "cascade-west-central"

In [38]:
raw = requests.get(url_template.format(region=ex_region))

In [39]:
soup = BeautifulSoup(raw.text, 'html5lib')

In [63]:
tn = soup.find(class_='forecast-detailed')
"\n".join(map(lambda x: x.text, tn.find_all('p')))

"A frontal band of rain\xa0will progress across the Cascade West slopes on Wednesday night,\xa0with moderate to occasionally strong southerly winds during the evening hours. Precipitation has started as rain in most of our forecasting terrain. Light to moderate rain and snow showers on a moderate SW flow and\xa0a significant cooling trend follow the frontal passage during the late evening hours. Thursday will feature continued gradual cooling with the\xa0showery pattern initiated Wednesday night\xa0and light to moderate SW winds.\xa0\nNote that a\xa0refreezing 1/17 rain crust will form Wednesday night and provide a bed surface for wind and storm slab avalanches on Thursday.\nCascade West-North above treeline terrain will have dangerous wind slab and storm slab on Thursday. The upper elevations of the above treeline terrain in this zone will not see all snow with the Wednesday night storm system, adding far more new snow relative other locations during the peak intensity of the storm. D

In [41]:
ta.select("div.span4")[1].select('h4.mobile')[0].text
ta.select("div.span3")[0].select('h4.mobile')[0].text

IndexError: list index out of range

In [42]:
def elevBandForecast(fcst, band):
    result = {}
    fcst = fcst.find(id=band)
    now_day = fcst.select("div.span4")[1].select('h4.mobile')[0].text
    now_fcst = fcst.select("div.span4")[1].select('div.danger-description h4')[0].text
    fut_day = fcst.select("div.span3")[0].select('h4.mobile')[0].text
    fut_fcst = fcst.select("div.span3")[0].select('div.danger-description h4')[0].text
    result['now']  = {
        'day': now_day,
        'danger' : now_fcst
    }
    result['tomorrow'] = {
        'day' : fut_day,
        'danger' : fut_fcst
    }
    return(result)

In [43]:
elevBandForecast(soup, 'treeline-above')

{'now': {'danger': 'Considerable', 'day': 'Thursday'},
 'tomorrow': {'danger': 'Considerable', 'day': 'Outlook for Fridayi'}}

In [44]:
def updatedTime(fcst):
    return(fcst.find('div', 'forecast-date').text.strip())

In [45]:
updatedTime(soup)

'Issued: 6:58 PM PST Wednesday, January 17, 2018'

In [46]:
def problems(fcst):
    result = []
    problems = fcst.find_all(class_="problem")
    for problem in problems:
        this = {}
        this['name'] = problem.select("div.problem-description h3")[0].text.strip()[:-1]
        this['rose_img_url'] = "https://www.nwac.us"+ problems[0].select('img.problem-octagon')[0].attrs['src']
        this['likelihood_img_url'] = problem.select('img.problem-likelihood')[0].attrs['src']
        this['size_img_url'] =  problem.select('img.problem-sizes')[0].attrs['src']
        result.append(this)
    return(result)
    

In [47]:
problems(soup)

[{'likelihood_img_url': 'https://d22fgw9k2fjwhz.cloudfront.net/media/images/likelihood_images/likelihood-33.png',
  'name': 'Wind Slab',
  'rose_img_url': 'https://www.nwac.us/avalanche-forecast/octagon/problem/8837.png',
  'size_img_url': 'https://d22fgw9k2fjwhz.cloudfront.net/media/images/size_images/size-12.png'},
 {'likelihood_img_url': 'https://d22fgw9k2fjwhz.cloudfront.net/media/images/likelihood_images/likelihood-22.png',
  'name': 'Storm Slabs',
  'rose_img_url': 'https://www.nwac.us/avalanche-forecast/octagon/problem/8837.png',
  'size_img_url': 'https://d22fgw9k2fjwhz.cloudfront.net/media/images/size_images/size-12.png'}]

In [72]:
def detailed_forecast(fcst):
    fd = fcst.find(class_='forecast-detailed')
    title = fd.find('h2').text[0:-1]
    return({
        'forecast_title' : title,
        'forecast' : "\n".join(map(lambda x: x.text, fd.find_all('p')))
    })


In [73]:
detailed_forecast(soup)

{'forecast': "A frontal band of rain\xa0will progress across the Cascade West slopes on Wednesday night,\xa0with moderate to occasionally strong southerly winds during the evening hours. Precipitation has started as rain in most of our forecasting terrain. Light to moderate rain and snow showers on a moderate SW flow and\xa0a significant cooling trend follow the frontal passage during the late evening hours. Thursday will feature continued gradual cooling with the\xa0showery pattern initiated Wednesday night\xa0and light to moderate SW winds.\xa0\nNote that a\xa0refreezing 1/17 rain crust will form Wednesday night and provide a bed surface for wind and storm slab avalanches on Thursday.\nCascade West-North above treeline terrain will have dangerous wind slab and storm slab on Thursday. The upper elevations of the above treeline terrain in this zone will not see all snow with the Wednesday night storm system, adding far more new snow relative other locations during the peak intensity of

In [74]:
def detailed_summary(fcst):
    ta = fcst.find(id='discussion')
    allp = ta.find_all('p')
    summary = ""
    it = 0
    text = ""
    while text != "Observations":
        summary += text + '\n'
        text = allp[it].text
        it += 1
    return summary.strip()

In [49]:
detailed_summary(soup)

'A trace to 2" of high density snow fell Wednesday morning on a warming trend as warm front lifted across the Cascades and should be well bonded to 2-4" of higher elevation snow that fell on Tuesday. These small amounts of recent snow should be melting and rounding from rainfall currently saturating the snowpack will into the above treeline terrain on Wednesday afternoon.\nRecent snowfall sits on well-drained, generally saturated snowpack with less saturation in the above treeline terrain, but multiple crusts extending into this terrain. There\xa0are no notable layers of concern at this time. Thus, focus your attention on new storm instabilities in these zones moving forward.'

In [50]:
import json

In [75]:
def getRegionalForecast(region):
    forecast = {}
    url = url_template.format(region=region)
    tree = BeautifulSoup(requests.get(url).text, 'html5lib')
    # Forecast Time
    forecast["updateTime"] = updatedTime(tree)
    # Elevation Band Danger Levels
    danger = []
    for elev_band in ["treeline-above", 'treeline-near', 'treeline-below']:
        danger.append({elev_band : elevBandForecast(tree, elev_band)})
    forecast['danger'] = danger
    # Problems
    forecast['problems'] = problems(tree)
    forecast['detailed_summary'] = detailed_summary(tree)
    forecast['detailed_forecast'] = detailed_forecast(tree)
    return(forecast)
    
    
        

In [76]:
getRegionalForecast('cascade-west-north-baker')

{'danger': [{'treeline-above': {'now': {'danger': 'High', 'day': 'Thursday'},
    'tomorrow': {'danger': 'Considerable', 'day': 'Outlook for Fridayi'}}},
  {'treeline-near': {'now': {'danger': 'Considerable', 'day': 'Thursday'},
    'tomorrow': {'danger': 'Considerable', 'day': 'Outlook for Fridayi'}}},
  {'treeline-below': {'now': {'danger': 'Considerable', 'day': 'Thursday'},
    'tomorrow': {'danger': 'Considerable', 'day': 'Outlook for Fridayi'}}}],
 'detailed_forecast': {'forecast': "A frontal band of rain\xa0will progress across the Cascade West slopes on Wednesday night,\xa0with moderate to occasionally strong southerly winds during the evening hours. Precipitation has started as rain in most of our forecasting terrain. Light to moderate rain and snow showers on a moderate SW flow and\xa0a significant cooling trend follow the frontal passage during the late evening hours. Thursday will feature continued gradual cooling with the\xa0showery pattern initiated Wednesday night\xa0and

In [ ]:
from Flask import jsonify